In [1]:
import numpy as np
from scipy.linalg import lu_factor, lu_solve
from halton_points import HaltonPoints
from expressions import *
import matplotlib.pyplot as plt

In [6]:
def test(a):
    #mdict = dict()
    for t in range(5):
        a+=t
        #mdict[t] = a
        yield t,a

dict(test(1))

{0: 1, 1: 2, 2: 4, 3: 7, 4: 11}

# One step method

In [40]:
class Onestepmethod(object):
    def __init__(self, t0, te, N, tol, Mb, npnts, poly_b, uh):
        #self.f = f
        self.t0 = t0
        self.interval = [t0, te]
        self.grid = np.linspace(t0, te, N+2)
        self.h = (te-t0)/(N+1)
        self.N = N
        self.tol = tol
        self.s = len(self.b)
        self.Mb = Mb
        self.npnts = npnts,
        self.poly_b = poly_b
        self.uh = uh#assembled_matrix(Mb=self.Mb, npnts=self.npnts, beta=2, c=1, poly_b=self.poly_b)
        self.y0 = self.uh.X_0()
        self.dm = self.y0.ndim
        self.y0 = self.y0[0]
        self.J = self.uh.J()

    def f(self, t, X0):
        for xn in self.uh.Mi:
            self.uh.x = xn
            yield uh.F_m(X0)[0]

    def step(self):
        ti, yi = self.grid[0], self.y0
        t_i = ti
        yield np.array([ti]), yi
            
        for ti in self.grid[1:]:
            yi = yi + self.h*self.phi(t_i, yi)
            t_i = ti
            yield np.array([ti]), yi

    def solve(self):
        #print(np.array(list(self.step()))[:,:,0])
        self.solution = np.array(list(self.step()))

# RK Implicit

In [41]:
class RungeKutta_implicit(Onestepmethod):
    def phi(self, ti, yi):
        '''
        Calculates the sum of b_j*Y_j in one step of the Runge-Kutta method with y_{n+1}= y_n + h*sum_{j=1}^s b_j *Y
        where j=1,2,...,s and s is the number of stages, b the notes and Y the stages values.

        Parameters:
        ----------------
        t0 = float, current timestep
        y0 = 1xm vector, the last solution y_n. Where m is the lenght of the IC y_0 of IVP.

        '''
        M = 10
        # Initial value Y'_0
        stageDer = np.vstack(self.s*[np.vstack(tuple(self.f(ti, yi)))])
        #J = nd.Jacobian(self.f)([t0, y0[0]])
        # J = nd.Jacobian(self.f, t0, y0)

        #J = np.array([-5])
        #J = np.array([[0, 1], [-1, 0]])
        J = self.J
        #J = np.array([[0, 1], [-9.8*np.cos(yi[0]), 0]])
        stageVal = self.phi_solve(ti, yi, stageDer, J, M)
        return np.dot(self.b, stageVal.reshape(self.s, self.dm))
        # -|-return np.array([np.dot(self.b, stageVal.reshape(self.s, self.dm)[:, j]) for j in range(self.dm)])

    def phi_solve(self, t0, y0, initVal, J, M):
        '''
        This function solves the sm x sm system
        F(Y_i)=0 by Newton method with initial guess initVal.

        Parameters:
        ______________
        t0 = float, current timestep.
        y0 = 1 x m vector, the last solution y_n.
            Where m is the length on the initial condition
            y_0 of the IVP.
        initVal = Initial guess for the Newton iteration.
        J = m x m matrix, the Jacobian matrix of f()
            evaluated in y_i
        M = Maximal number of Newton iterations

        Returns:
        ______________
        The stage derivative Y'_i               
        '''
        JJ = np.eye(self.s * self.dm) - self.h * np.kron(self.A, J)
        luFactor = lu_factor(JJ)

        for i in range(M):
            initVal, norm_d = self.phi_newtonstep(t0, y0, initVal, luFactor)
            if norm_d < self.tol:
                print('Newton converged in {} steps'.format(i))
            elif i == M-1:
                raise ValueError('The Newton iteration did not converge.')

        return initVal

    def phi_newtonstep(self, t0, y0, initVal, luFactor):
        '''
        Takes one Newton step by solving
            G'(Y_i)(Y^{n+1}_{i} - Y^{n}_{i}) = -G(Y_i)

        where,
            G(Y_i) = Y_i - y_n - h * sum(a_{ij} * Y'_j)
                for j = 1,...,s

        Parameters:
        ______________
        t0 = 
        y0 =
        '''

        d = lu_solve(luFactor, -self.F(initVal.flatten(), t0, y0))

        return initVal.flatten() + d, np.linalg.norm(d)

    def F(self, stageDer, t_n, y_i):
        '''
        Returns the substraction Y'_i-
        '''
        stageDer_new = np.empty((self.s, self.dm))
        # print(stageDer.shape)

        for i in range(self.s):

            stageVal = y_i + self.h * np.dot(
                self.A[i, :], stageDer.reshape(self.s, self.dm))
            # -|-stageVal = y_i + np.array([self.h * np.dot(
            #     self.A[i, :], stageDer.reshape(self.s, self.dm)[:, j]) for j in range(self.dm)])

            stageDer_new[i, :] = self.f(t_n + self.c[i] * self.h, stageVal)

        return stageDer - stageDer_new.reshape(-1)


# SDIRK

In [42]:
class SDIRK(RungeKutta_implicit):
    def phi_solve(self, t0, y0, initVal, J, M):
        '''
        This function solves F(Y_i)=0
        '''
        #print('phi_solve')
        alpha = self.A[0, 0]

        JJ = np.eye(self.dm) - self.h * alpha * J

        luFactor = lu_factor(JJ)

        for i in range(M):
            initVal, norm_d = self.phi_newtonstep(
                t0, y0, initVal, J, luFactor)
            if norm_d < self.tol:
                print('Newton converged in {} steps'.format(i))
                break
            elif i == M-1:
                raise ValueError('The Newton iteration did not converge.')

        return initVal

    def phi_newtonstep(self, t0, y0, initVal, J, luFactor):
        '''
        Takes on Newton step by solving
        '''
        #print('phi_newtonstep')
        x = []
        for i in range(self.s):
            rhs = -self.F(initVal.flatten(), t0, y0)[i * self.dm:(i+1) * self.dm] + np.sum(
                [self.h * self.A[i, j] * np.dot(J, x[j]) for j in range(i)], axis=0)
            d = lu_solve(luFactor, rhs)
            x.append(d)

        return initVal + x, np.linalg.norm(x)

# Gauss

In [43]:
class Gauss(RungeKutta_implicit):
    A = np. array([
        [5/36, 2/9 - np.sqrt(15)/15, 5/36 - np.sqrt(15)/30],
        [5/36 + np.sqrt(15)/24, 2/9, 5/36 - np.sqrt(15)/24],
        [5/36 + np.sqrt(15)/30, 2/9 + np.sqrt(15)/15, 5/36]
    ])
    b = np.array([5/18, 4/9, 5/18])
    c = np.array([1/2-np.sqrt(15)/10, 1/2, 1/2 + np.sqrt(15)/10])

# SDIRK 2 stages

In [44]:
class SDIRK_tableau2s(SDIRK):
    p = (3 - np.sqrt(3))/6
    A = np.array([[p, 0], [1 - 2*p, p]])
    b = np.array([1/2, 1/2])
    c = np.array([p, 1-p])


# SDIRK 5 stages

In [45]:
class SDIRK_tableau5s(SDIRK):
    A = np.array([
        [1/4, 0, 0, 0, 0],
        [1/2, 1/4, 0, 0, 0],
        [17/50, -1/25, 1/4, 0, 0],
        [371/1360, -137/2720, 15/544, 1/4, 0],
        [25/24, -49/48, 125/16, -85/12, 1/4]
    ])
    b = np.array([25/24, -49/48, 125/16, -85/12, 1/4])
    c = np.array([1/4, 3/4, 11/20, 1/2, 1])

# Test scalar

In [46]:
# t0, te = 0, 1.
# tol_newton = 1e-9
# tol_sol = 1e-5
# N = 20

# y0 = np.array([1])
# def f(t, y): return -5*y

# scalar = Gauss(f, y0, t0, te, N, tol_newton)

# scalar.solve()
# S = scalar.solution
# t = S[:, 0]
# y = S[:, 1]
# a = np.exp((-5*t))
# error = np.abs(y-a)

# plt.plot(t, error)
# plt.show()

# Test system

In [47]:
# # np.vstack(y)
# t0, te = 0, 1.
# tol_newton = 1e-9
# tol_sol = 1e-5
# N = 500

# y0 = np.array([2., 3.])
# M = np.array([[0, 1], [-1, 0]])
# def f(t,y): return np.dot(M, y)
# system = SDIRK_tableau2s(f, y0, t0, te, N, tol_newton)


# timegrid  = np.linspace(t0, te, N+2)
# def exact_sol(timegrid):
#     for t in timegrid:
#         y1 = np.array([2*np.cos(t) + 3*np.sin(t)])
#         y2 = np.array([-2*np.sin(t)+ 3*np.cos(t)])
#         yield np.hstack((y1,y2))

# system.solve()
# S = system.solution
# t = S[:, 0]
# y = S[:, 1]
# a = np.vstack(tuple(exact_sol(timegrid)))
# error = np.linalg.norm(np.vstack(y)-a, axis=-1)
# plt.plot(np.vstack(t), error)
# plt.show()

# Test thesis

In [48]:
Mb = np.array([
    [0., 0.],
    [0., 0.5],
    [0., 1.],
    [0.5, 1.],
    [1., 1.],
    [1., 0.5],
    [1., 0.],
    [0.5, 0.]
])

poly_b = np.array([[-1, -1, 1], [1/2, 3/2, -1], [3/2, 1/8, -3/8]])
x = np.array([0.16, .093])
npnts = 1

t0, te = 0, 1.
tol_newton = 1e-9
tol_sol = 1e-5
N = 500

uh = assembled_matrix(Mb, npnts, beta=2, c=1, poly_b=poly_b)

In [49]:
# def Fm(uh, X0, ti=0):
#     for xn in uh.Mi:
#         uh.x = xn
#         yield uh.F_m(X0)[0], uh.J(X0)[0]

In [50]:
#(self, t0, te, N, tol, Mb, npnts, poly_b):

In [51]:
# 
#(self, t0, te, N, tol, Mb, npnts, poly_b):

system = SDIRK_tableau2s(t0=t0, te=te, N=N, tol=tol_newton, Mb=Mb, npnts=npnts, poly_b=poly_b, uh=uh)


# timegrid  = np.linspace(t0, te, N+2)
# def exact_sol(timegrid):
#     for t in timegrid:
#         y1 = np.array([2*np.cos(t) + 3*np.sin(t)])
#         y2 = np.array([-2*np.sin(t)+ 3*np.cos(t)])
#         yield np.hstack((y1,y2))

system.solve()
# S = system.solution
# t = S[:, 0]
# y = S[:, 1]
# a = np.vstack(tuple(exact_sol(timegrid)))
# error = np.linalg.norm(np.vstack(y)-a, axis=-1)
# plt.plot(np.vstack(t), error)
# plt.show()

ValueError: cannot reshape array of size 2 into shape (1,)

In [17]:
assembled_matrix(Mb=Mb, npnts=npnts, beta=2, c=1, poly_b=poly_b).Mi

array([[0.5       , 0.33333333]])